### Imports and loading 

In [ ]:
import asyncio
import dspy
from prompt_optimization.seed_prompts import ATOMIZER_PROMPT, PLANNER_PROMPT, AGGREGATOR_PROMPT, ATOMIZER_DEMOS, PLANNER_DEMOS
from dspy import GEPA

from prompt_optimization import (
    get_default_config,
    LMConfig,
    patch_romaconfig,
    load_aimo_datasets,
    ComponentJudge,
    MetricWithFeedback,
    create_optimizer,
)
from prompt_optimization.seed_prompts import (
    ATOMIZER_PROMPT,
    ATOMIZER_DEMOS,
    PLANNER_PROMPT,
    PLANNER_DEMOS,
    AGGREGATOR_PROMPT,
)
from prompt_optimization.grader_prompt import GRADER_PROMPT
from roma_dspy.config import load_config
from roma_dspy.core.engine.solve import RecursiveSolver
from roma_dspy.core.modules.recursive_solver import RecursiveSolverModule
from roma_dspy.utils import AsyncParallelExecutor

dspy.settings.provide_traceback = True  # optional but mirrors the old notebook
opt_cfg = load_config(profile="test")

ImportError: cannot import name 'GRADER_PROMPT' from 'prompt_optimization.seed_prompts' (/Users/salahalzubi/cursor_projects/ROMA-DSPy/prompt_optimization/seed_prompts/__init__.py)

### Config LLMS

In [11]:
# Batch the knobs you used to tweak in the notebook.
opt_cfg.train_size = 32
opt_cfg.val_size = 8
opt_cfg.test_size = 8
opt_cfg.dataset_seed = 42
opt_cfg.max_metric_calls = 225
opt_cfg.max_depth = 1
opt_cfg.enable_logging = False

In [12]:
#Add few-shot examples + prompts
opt_cfg.agents.atomizer.signature_instructions = ATOMIZER_PROMPT
opt_cfg.agents.planner.signature_instructions = PLANNER_PROMPT
opt_cfg.agents.aggregator.signature_instructions = AGGREGATOR_PROMPT

### Init solvers and what not

In [13]:
solver = RecursiveSolver(
    config=opt_cfg,
    max_depth=opt_cfg.max_depth,
    enable_logging=opt_cfg.enable_logging,
    enable_checkpoints=False,
)
solver_module = RecursiveSolverModule(solver=solver)

2025-10-17 11:00:26.553 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature for atomizer
2025-10-17 11:00:26.554 | INFO     | roma_dspy.core.factory.agent_factory:create_agent:91 - Created atomizer agent (task_type=default, signature=default)
2025-10-17 11:00:26.556 | DEBUG    | roma_dspy.core.registry.agent_registry:register_agent:180 - Registered atomizer instance #15 (task_type=default)
2025-10-17 11:00:26.557 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature for planner
2025-10-17 11:00:26.559 | INFO     | roma_dspy.core.factory.agent_factory:create_agent:91 - Created planner agent (task_type=default, signature=default)
2025-10-17 11:00:26.559 | DEBUG    | roma_dspy.core.registry.agent_registry:register_agent:180 - Registered planner instance #16 (task_type=default)
2025-10-17 11:00:26.560 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:119 - Using default signature 

In [ ]:
judge_lm = LMConfig("openrouter/openai/gpt-5", temperature=1.0, max_tokens=128000, cache=True)


In [ ]:
judge = ComponentJudge(prompt=GRADER_PROMPT, lm_config=judge_lm)  # keyword required after the refactor
metric = MetricWithFeedback(judge)

AttributeError: 'ROMAConfig' object has no attribute 'judge_lm'

In [ ]:
train_set, val_set, test_set = load_aimo_datasets(
    train_size=opt_cfg.train_size,
    val_size=opt_cfg.val_size,
    test_size=opt_cfg.test_size,
    seed=opt_cfg.dataset_seed,
)

### Perform an eval on the val set

In [ ]:
# executor = AsyncParallelExecutor(max_concurrency=4)

# results = await executor.execute_batch(solver_module, test_set)

In [ ]:
# print(results)

### Prompt tuning stuff

In [ ]:
optimizer = GEPA(
    metric=metric,
    # auto="light",
    component_selector="round_robin",
    max_metric_calls=24,
    add_format_failure_as_feedback=True,
    num_threads=6,
    track_stats=True,
    log_dir="logs/aime_test",
    # use_wandb=True,
    # wandb_init_kwargs={"project": "aime_test"},
    reflection_minibatch_size=8,
    reflection_lm=dspy.LM(model="openrouter/anthropic/claude-sonnet-4.5", temperature=.75, max_tokens=128000)
)

In [ ]:
optimized_program = optimizer.compile(
    solver_module,
    trainset=train_set,
    valset=val_set,
)

In [ ]:
print(optimized_program.named_predictors()[1][1].signature.instructions)